In [1]:
import os
import zstandard
import polars as pl
import requests
import json
import tempfile
import re

In [2]:
def download_process_zst(url:str) -> pl.DataFrame:    
    # Create a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        # Download
        response = requests.get(url)
        zst_path = os.path.join(temp_dir, "data.zst")
        with open(zst_path, 'wb') as f:
            f.write(response.content)

        # Decompress
        json_path = os.path.join(temp_dir, "data.jsonl")
        with open(zst_path, 'rb') as compressed_file:
            with open(json_path, 'wb') as decompressed_file:
                dctx = zstandard.ZstdDecompressor()
                dctx.copy_stream(compressed_file, decompressed_file)

        # File is in JSONL format
        df = pl.read_ndjson(
            json_path,
            infer_schema_length=10000,
            ignore_errors=True
        )
        
    return df    

print("Downloading and processing data...")
df = download_process_zst("https://the-eye.eu/redarcs/files/truerateme_submissions.zst")
print(df.columns)

['media', 'secure_media_embed', 'author_flair_type', 'link_flair_type', 'pwls', 'send_replies', 'media_only', 'media_embed', 'removal_reason', 'subreddit', 'hidden', 'post_categories', 'whitelist_status', 'link_flair_css_class', 'wls', 'score', 'num_reports', 'over_18', 'author', 'author_flair_richtext', 'contest_mode', 'approved_by', 'author_flair_css_class', 'author_flair_template_id', 'created', 'locked', 'name', 'permalink', 'selftext', 'is_self', 'distinguished', 'subreddit_type', 'suggested_sort', 'thumbnail', 'num_crossposts', 'hide_score', 'is_video', 'link_flair_text_color', 'num_comments', 'title', 'id', 'can_gild', 'author_flair_background_color', 'author_flair_text_color', 'likes', 'domain', 'link_flair_text', 'selftext_html', 'user_reports', 'clicked', 'url', 'report_reasons', 'thumbnail_height', 'is_original_content', 'downs', 'rte_mode', 'secure_media', 'parent_whitelist_status', 'is_crosspostable', 'pinned', 'archived', 'no_follow', 'gilded', 'category', 'created_utc', 

In [3]:
# select only needed columns
df = df.select([
    "id", "author", "created_utc", "subreddit",         # metadata
    "title", "selftext", "media_embed", "media", "url", # content
])
print(df.head(5))

shape: (5, 9)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ id     ┆ author     ┆ created_ut ┆ subreddit ┆ … ┆ selftext  ┆ media_emb ┆ media     ┆ url       │
│ ---    ┆ ---        ┆ c          ┆ ---       ┆   ┆ ---       ┆ ed        ┆ ---       ┆ ---       │
│ str    ┆ str        ┆ ---        ┆ str       ┆   ┆ str       ┆ ---       ┆ struct[3] ┆ str       │
│        ┆            ┆ i64        ┆           ┆   ┆           ┆ struct[5] ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 6twkns ┆ Hereforthe ┆ 1502825794 ┆ trueratem ┆ … ┆ [removed] ┆ {null,nul ┆ null      ┆ https://w │
│        ┆ kek        ┆            ┆ e         ┆   ┆           ┆ l,null,nu ┆           ┆ ww.reddit │
│        ┆            ┆            ┆           ┆   ┆           ┆ ll,null}  ┆           ┆ .com/r/tr │
│        ┆            ┆            ┆           ┆   ┆           ┆           ┆ 

In [4]:
# remove empty, removed, and delted posts
df = df.filter(
    (pl.col("media_embed").is_not_null()) &
    (pl.col("media").is_not_null()) &
    (pl.col("url").is_not_null()) &
    (pl.col("selftext") != "[removed]") &
    (pl.col("selftext") != "[deleted]")
)
df

id,author,created_utc,subreddit,title,selftext,media_embed,media,url
str,str,i64,str,str,str,struct[5],struct[3],str
"""6um2bk""","""Josh_norman05""",1503103734,"""truerateme""","""[21M] Rateme, I'm 6'1"" and 200…","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2Fv2xU7%2Fembed&amp;url=http%3A%2F%2Fimgur.com%2Fa%2Fv2xU7&amp;image=http%3A%2F%2Fi.imgur.com%2FpCLxBv8.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",550,null,false,550}","{{null,""Imgur: The most awesome images on the Internet."",550,""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2Fv2xU7%2Fembed&amp;url=http%3A%2F%2Fimgur.com%2Fa%2Fv2xU7&amp;image=http%3A%2F%2Fi.imgur.com%2FpCLxBv8.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",""Imgur"",""http://imgur.com"",315,""http://i.imgur.com/pCLxBv8.jpg?fb"",600,""Imgur: The most awesome images on the Internet"",""rich"",null,""1.0"",550,null,null},""imgur.com"",null}","""http://imgur.com/a/v2xU7"""
"""6umpik""","""PettingXu""",1503111646,"""truerateme""","""M23 tear me a new one guys""","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2F2loQ6%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2F2loQ6&amp;image=http%3A%2F%2Fi.imgur.com%2F7HP3ftQ.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",550,null,false,550}","{{null,""Imgur: The most awesome images on the Internet."",550,""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2F2loQ6%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2F2loQ6&amp;image=http%3A%2F%2Fi.imgur.com%2F7HP3ftQ.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",""Imgur"",""http://imgur.com"",315,""http://i.imgur.com/7HP3ftQ.jpg?fb"",600,""Reddit Rateme"",""rich"",null,""1.0"",550,null,null},""m.imgur.com"",null}","""https://m.imgur.com/a/2loQ6"""
"""6un283""","""Mr-Rushifa""",1503116342,"""truerateme""","""[22M] 6'6"" Was told to post he…","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2FD0rJP%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2FD0rJP&amp;image=http%3A%2F%2Fi.imgur.com%2FHkIQCm5.jpg%3Ffb&amp;key=2aa3c4d5f3de4f5b9120b660ad850dc9&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",550,null,false,550}","{{null,""Imgur: The most awesome images on the Internet."",550,""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2FD0rJP%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2FD0rJP&amp;image=http%3A%2F%2Fi.imgur.com%2FHkIQCm5.jpg%3Ffb&amp;key=2aa3c4d5f3de4f5b9120b660ad850dc9&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",""Imgur"",""http://imgur.com"",315,""http://i.imgur.com/HkIQCm5.jpg?fb"",600,""Gallery"",""rich"",null,""1.0"",550,null,null},""imgur.com"",null}","""https://imgur.com/a/D0rJP"""
"""6un2v8""","""mindmonkey00""",1503116572,"""truerateme""","""[M 23] Rate my ginger ass and …","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2FRa0M6%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2FRa0M6&amp;image=http%3A%2F%2Fi.imgur.com%2FESsgdIW.jpg%3Ffb&amp;key=522b

In [5]:
df['media'][0]

{'oembed': {'cache_age': None,
  'description': 'Imgur: The most awesome images on the Internet.',
  'height': 550,
  'html': '&lt;iframe class="embedly-embed" src="//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2Fv2xU7%2Fembed&amp;url=http%3A%2F%2Fimgur.com%2Fa%2Fv2xU7&amp;image=http%3A%2F%2Fi.imgur.com%2FpCLxBv8.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur" width="550" height="550" scrolling="no" frameborder="0" allowfullscreen&gt;&lt;/iframe&gt;',
  'provider_name': 'Imgur',
  'provider_url': 'http://imgur.com',
  'thumbnail_height': 315,
  'thumbnail_url': 'http://i.imgur.com/pCLxBv8.jpg?fb',
  'thumbnail_width': 600,
  'title': 'Imgur: The most awesome images on the Internet',
  'type': 'rich',
  'url': None,
  'version': '1.0',
  'width': 550,
  'author_name': None,
  'author_url': None},
 'type': 'imgur.com',
 'reddit_video': None}

In [6]:
df['media'][0]['oembed']['thumbnail_url']

'http://i.imgur.com/pCLxBv8.jpg?fb'

In [14]:
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def download_thumbnail(url: str, path: str, max_retries=5):
    # Configure retry strategy with backoff
    retry_strategy = Retry(
        total=max_retries,
        backoff_factor=1,  # Exponential backoff: 1, 2, 4, 8, 16 seconds between retries
        status_forcelist=[429, 500, 502, 503, 504],  # Retry on these status codes
        allowed_methods=["GET"]
    )
    
    # Create a session with the retry strategy
    session = requests.Session()
    session.mount("https://", HTTPAdapter(max_retries=retry_strategy))
    session.mount("http://", HTTPAdapter(max_retries=retry_strategy))
    
    # Add headers to make request more browser-like
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'image/webp,image/apng,image/*,*/*;q=0.8',
        'Referer': 'https://imgur.com/'
    }
    
    # Use the session to get the image with retries, redirects and headers
    response = session.get(url, headers=headers, allow_redirects=True, timeout=30)
    response.raise_for_status()
    
    # Create directory if needed
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Write image to file
    with open(path, 'wb') as f:
        f.write(response.content)

def remove_url_args(url:str):
    return re.sub(r'\?.*$', '', url)

def is_img_url(url:str):
    return url.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.webp', '.bmp', '.tiff', '.svg'))

In [15]:
is_img_url(remove_url_args(df[0]['media'][0]['oembed']['thumbnail_url']))

True

In [16]:
df[0]['media'][0]

{'oembed': {'cache_age': None,
  'description': 'Imgur: The most awesome images on the Internet.',
  'height': 550,
  'html': '&lt;iframe class="embedly-embed" src="//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2Fv2xU7%2Fembed&amp;url=http%3A%2F%2Fimgur.com%2Fa%2Fv2xU7&amp;image=http%3A%2F%2Fi.imgur.com%2FpCLxBv8.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur" width="550" height="550" scrolling="no" frameborder="0" allowfullscreen&gt;&lt;/iframe&gt;',
  'provider_name': 'Imgur',
  'provider_url': 'http://imgur.com',
  'thumbnail_height': 315,
  'thumbnail_url': 'http://i.imgur.com/pCLxBv8.jpg?fb',
  'thumbnail_width': 600,
  'title': 'Imgur: The most awesome images on the Internet',
  'type': 'rich',
  'url': None,
  'version': '1.0',
  'width': 550,
  'author_name': None,
  'author_url': None},
 'type': 'imgur.com',
 'reddit_video': None}

In [17]:
# Save df
df.write_parquet("reddit_posts.parquet")

In [19]:
download_thumbnail('https://i.imgur.com/t7QQzTW.jpg', './temp.jpg')

In [21]:
# Download all thumbnails in df, updating rows with the local path upon download
# note that we write every step to parquet to avoid losing data
# also we skip rows if file already exists
if 'local_thumbnail_path' not in df.columns:
    df = df.with_columns(pl.lit("").alias('local_thumbnail_path'))

df_view = df.clone()
for idx, row in enumerate(df_view.iter_rows(named=True)):
    # Extract thumbnail url
    try:
        thumbnail_url = row['media']['oembed']['thumbnail_url']
    except (KeyError, TypeError):
        continue
    if thumbnail_url == None: continue
    
    clean_url = remove_url_args(thumbnail_url)
    if not is_img_url(clean_url):  continue
    
    ext = os.path.splitext(clean_url)[1]
    local_path = os.path.join("thumbnails", f"{row['id']}{ext}")
    
    if os.path.exists(local_path): continue
    
    try:
        download_thumbnail(clean_url, local_path)
        df[idx, 'local_thumbnail_path'] = local_path
        df.write_parquet("reddit_posts.parquet")
    except:
        print(f"Failed to download {clean_url}")
        continue

df

Failed to download https://d1wst0behutosd.cloudfront.net/videos/13470266/thumb.jpg
Failed to download https://i.ytimg.com/vi/gjr52rhaXuY/hqdefault.jpg
Failed to download https://thumbs.gfycat.com/FaintFrightenedGemsbuck-size_restricted.gif
Failed to download https://i.ytimg.com/vi/4XAYjZapSiA/hqdefault.jpg
Failed to download https://i.ytimg.com/vi/9vDG4xWZM3Q/hqdefault.jpg
Failed to download https://cdn-b-east.streamable.com/image/ekz8c.jpg
Failed to download https://thumbs.gfycat.com/MajesticFabulousIrishsetter-size_restricted.gif
Failed to download https://cdn-b-east.streamable.com/image/dev4l.jpg
Failed to download https://i.ytimg.com/vi/dyTe7jOF6yM/hqdefault.jpg
Failed to download https://i.ytimg.com/vi/VBuXYAj8up4/hqdefault.jpg
Failed to download https://i.ytimg.com/vi/IAw8XtzKKqM/hqdefault.jpg
Failed to download https://thumbs.gfycat.com/SmugKindheartedBluewhale-size_restricted.gif
Failed to download https://i.ytimg.com/vi/8WIVqq0vqp0/hqdefault.jpg
Failed to download https://i.yt

id,author,created_utc,subreddit,title,selftext,media_embed,media,url,local_thumbnail_path
str,str,i64,str,str,str,struct[5],struct[3],str,str
"""6um2bk""","""Josh_norman05""",1503103734,"""truerateme""","""[21M] Rateme, I'm 6'1"" and 200…","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2Fv2xU7%2Fembed&amp;url=http%3A%2F%2Fimgur.com%2Fa%2Fv2xU7&amp;image=http%3A%2F%2Fi.imgur.com%2FpCLxBv8.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",550,null,false,550}","{{null,""Imgur: The most awesome images on the Internet."",550,""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2Fv2xU7%2Fembed&amp;url=http%3A%2F%2Fimgur.com%2Fa%2Fv2xU7&amp;image=http%3A%2F%2Fi.imgur.com%2FpCLxBv8.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",""Imgur"",""http://imgur.com"",315,""http://i.imgur.com/pCLxBv8.jpg?fb"",600,""Imgur: The most awesome images on the Internet"",""rich"",null,""1.0"",550,null,null},""imgur.com"",null}","""http://imgur.com/a/v2xU7""","""thumbnails/6um2bk.jpg"""
"""6umpik""","""PettingXu""",1503111646,"""truerateme""","""M23 tear me a new one guys""","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2F2loQ6%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2F2loQ6&amp;image=http%3A%2F%2Fi.imgur.com%2F7HP3ftQ.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",550,null,false,550}","{{null,""Imgur: The most awesome images on the Internet."",550,""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2F2loQ6%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2F2loQ6&amp;image=http%3A%2F%2Fi.imgur.com%2F7HP3ftQ.jpg%3Ffb&amp;key=522baf40bd3911e08d854040d3dc5c07&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",""Imgur"",""http://imgur.com"",315,""http://i.imgur.com/7HP3ftQ.jpg?fb"",600,""Reddit Rateme"",""rich"",null,""1.0"",550,null,null},""m.imgur.com"",null}","""https://m.imgur.com/a/2loQ6""","""thumbnails/6umpik.jpg"""
"""6un283""","""Mr-Rushifa""",1503116342,"""truerateme""","""[22M] 6'6"" Was told to post he…","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2FD0rJP%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2FD0rJP&amp;image=http%3A%2F%2Fi.imgur.com%2FHkIQCm5.jpg%3Ffb&amp;key=2aa3c4d5f3de4f5b9120b660ad850dc9&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",550,null,false,550}","{{null,""Imgur: The most awesome images on the Internet."",550,""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2FD0rJP%2Fembed&amp;url=https%3A%2F%2Fimgur.com%2Fa%2FD0rJP&amp;image=http%3A%2F%2Fi.imgur.com%2FHkIQCm5.jpg%3Ffb&amp;key=2aa3c4d5f3de4f5b9120b660ad850dc9&amp;type=text%2Fhtml&amp;schema=imgur"" width=""550"" height=""550"" scrolling=""no"" frameborder=""0"" allowfullscreen&gt;&lt;/iframe&gt;"",""Imgur"",""http://imgur.com"",315,""http://i.imgur.com/HkIQCm5.jpg?fb"",600,""Gallery"",""rich"",null,""1.0"",550,null,null},""imgur.com"",null}","""https://imgur.com/a/D0rJP""","""thumbnails/6un283.jpg"""
"""6un2v8""","""mindmonkey00""",1503116572,"""truerateme""","""[M 23] Rate my ginger ass and …","""""","{""&lt;iframe class=""embedly-embed"" src=""//cdn.embedly.com/widgets/media.html?src=%2F%2Fimgur.com%2Fa%2FRa0M6%2Fembed&a